# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 8 2023 10:22AM,258936,10,SNAP35289165,"Snap Medical Industries, LLC",Released
1,Mar 8 2023 10:22AM,258936,10,SNAP37231,"Snap Medical Industries, LLC",Released
2,Mar 8 2023 10:15AM,258897,15,PNC432566A,"Person & Covey, Inc.",Released
3,Mar 8 2023 10:15AM,258897,15,PNC432567A,"Person & Covey, Inc.",Released
4,Mar 8 2023 10:15AM,258897,15,PNC432573A,"Person & Covey, Inc.",Released
5,Mar 8 2023 10:15AM,258897,15,PNC432563A,"Person & Covey, Inc.",Released
6,Mar 8 2023 10:15AM,258896,15,30014603,"Alliance Pharma, Inc.",Released
7,Mar 8 2023 10:15AM,258896,15,30014604,"Alliance Pharma, Inc.",Released
8,Mar 8 2023 10:15AM,258896,15,30014605,"Alliance Pharma, Inc.",Released
9,Mar 8 2023 10:15AM,258896,15,30014608,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,258931,Released,1
31,258933,Released,1
32,258934,Released,4
33,258935,Released,1
34,258936,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258931,NaN,NaN,1.0
258933,NaN,NaN,1.0
258934,NaN,NaN,4.0
258935,NaN,NaN,1.0
258936,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258712,0.0,0.0,37.0
258759,0.0,1.0,30.0
258780,0.0,0.0,16.0
258811,12.0,3.0,0.0
258816,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258712,0,0,37
258759,0,1,30
258780,0,0,16
258811,12,3,0
258816,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258712,0,0,37
1,258759,0,1,30
2,258780,0,0,16
3,258811,12,3,0
4,258816,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258712,,,37
1,258759,,1,30
2,258780,,,16
3,258811,12,3,
4,258816,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC"
2,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc."
6,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc."
11,Mar 8 2023 10:14AM,258935,10,"Nextsource Biotechnology, LLC"
12,Mar 8 2023 9:56AM,258934,10,Eywa Pharma Inc.
16,Mar 8 2023 9:39AM,258780,15,"Person & Covey, Inc."
32,Mar 8 2023 9:38AM,258933,15,Lonza
33,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc."
64,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc."
101,Mar 8 2023 9:31AM,258931,10,"CLINUVEL, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC",,,2
1,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",,,4
2,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",,,5
3,Mar 8 2023 10:14AM,258935,10,"Nextsource Biotechnology, LLC",,,1
4,Mar 8 2023 9:56AM,258934,10,Eywa Pharma Inc.,,,4
5,Mar 8 2023 9:39AM,258780,15,"Person & Covey, Inc.",,,16
6,Mar 8 2023 9:38AM,258933,15,Lonza,,,1
7,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc.",,1,30
8,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc.",,,37
9,Mar 8 2023 9:31AM,258931,10,"CLINUVEL, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC",2,,
1,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",4,,
2,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",5,,
3,Mar 8 2023 10:14AM,258935,10,"Nextsource Biotechnology, LLC",1,,
4,Mar 8 2023 9:56AM,258934,10,Eywa Pharma Inc.,4,,
5,Mar 8 2023 9:39AM,258780,15,"Person & Covey, Inc.",16,,
6,Mar 8 2023 9:38AM,258933,15,Lonza,1,,
7,Mar 8 2023 9:33AM,258759,15,"Alliance Pharma, Inc.",30,1,
8,Mar 8 2023 9:33AM,258712,15,"Person & Covey, Inc.",37,,
9,Mar 8 2023 9:31AM,258931,10,"CLINUVEL, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC",2,,
1,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",4,,
2,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",5,,
3,Mar 8 2023 10:14AM,258935,10,"Nextsource Biotechnology, LLC",1,,
4,Mar 8 2023 9:56AM,258934,10,Eywa Pharma Inc.,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC",2.0,NaN,NaN
1,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",4.0,NaN,NaN
2,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",5.0,NaN,NaN
3,Mar 8 2023 10:14AM,258935,10,"Nextsource Biotechnology, LLC",1.0,NaN,NaN
4,Mar 8 2023 9:56AM,258934,10,Eywa Pharma Inc.,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 8 2023 10:22AM,258936,10,"Snap Medical Industries, LLC",2.0,0.0,0.0
1,Mar 8 2023 10:15AM,258897,15,"Person & Covey, Inc.",4.0,0.0,0.0
2,Mar 8 2023 10:15AM,258896,15,"Alliance Pharma, Inc.",5.0,0.0,0.0
3,Mar 8 2023 10:14AM,258935,10,"Nextsource Biotechnology, LLC",1.0,0.0,0.0
4,Mar 8 2023 9:56AM,258934,10,Eywa Pharma Inc.,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3365888,38.0,10.0,12.0
12,1812266,5.0,2.0,0.0
15,1552977,93.0,1.0,0.0
19,776645,3.0,23.0,0.0
21,517809,2.0,0.0,0.0
22,258912,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3365888,38.0,10.0,12.0
1,12,1812266,5.0,2.0,0.0
2,15,1552977,93.0,1.0,0.0
3,19,776645,3.0,23.0,0.0
4,21,517809,2.0,0.0,0.0
5,22,258912,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,38.0,10.0,12.0
1,12,5.0,2.0,0.0
2,15,93.0,1.0,0.0
3,19,3.0,23.0,0.0
4,21,2.0,0.0,0.0
5,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,38.0
1,12,Released,5.0
2,15,Released,93.0
3,19,Released,3.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21,22
Status,,,,,,
Completed,12.0,0.0,0.0,0.0,0.0,0.0
Executing,10.0,2.0,1.0,23.0,0.0,1.0
Released,38.0,5.0,93.0,3.0,2.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21,22
0,Completed,12.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,2.0,1.0,23.0,0.0,1.0
2,Released,38.0,5.0,93.0,3.0,2.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21,22
0,Completed,12.0,0.0,0.0,0.0,0.0,0.0
1,Executing,10.0,2.0,1.0,23.0,0.0,1.0
2,Released,38.0,5.0,93.0,3.0,2.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()